In [1]:
from conf import REDDIT_API_CONFIG as api_config
from conf import REDDIT_USER_CONFIG as user_config

# OAuth Requests

source: [How-to Use The Reddit API in Python](https://www.youtube.com/watch?v=FdjVoOf9HN4)

In [2]:
import requests

In [3]:
auth = requests.auth.HTTPBasicAuth(api_config["CLIENT_ID"], api_config["SECRET_KEY"])

In [4]:
headers = {
    'User-Agent': 'BDA_API/0.0.2'
}

In [5]:
req = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, 
                    data=user_config, 
                    headers=headers)

In [6]:
req.json()

{'access_token': '2378206320929-EypR31HObyMpMx1EP6oEkqO1M0P-8w',
 'token_type': 'bearer',
 'expires_in': 86400,
 'scope': '*'}

In [7]:
TOKEN = req.json()['access_token']

In [8]:
headers["Authorization"] = f'bearer {TOKEN}'

### "Hot" Posts

In [18]:
subreddit = "bitcoin"
listing = "hot"
timeframe = "hour"

In [19]:
req = requests.get(f"https://oauth.reddit.com/r/{subreddit}/{listing}.json?t={timeframe}", headers=headers)
req

<Response [200]>

In [20]:
# req.json()

In [21]:
for post in req.json()['data']['children']:
    print(post['data']['title'])

Daily Discussion, November 21, 2022
Mentor Monday, November 21, 2022: Ask all your bitcoin questions!
Why are humans so naive?
Tucker on Monetary Printing Presses
Too soon?? New Bitcoin piece features a live ticker. @danielallencohen
These signs advocating for Bitcoin vs CBDCs are up in multiple locations
Three BTC cycles, same story with my friends: they ape in on ATH, I tell them it's not the time &amp; I will say when. They are like ”thx, can't wait”, but when I call them on dips they chicken out. What's the matter with people?
It should be obvious by now, Bitcoin, not blockchain. Bitcoin, not crypto. Bitcoin, not DeFi. Bitcoin, not Web3. Bitcoin, not CBDCs. Bitcoin, not centralized neo-banks that sell you fractional-reserve paper bitcoin.
Few understand...
The plan is to hold bitcoin forever, right?
Bitcoin Survived Mt. Gox, FTX is "nothing"
The absolute scarcity of bitcoin combined with the fractional reserve games played by major banks, exchanges, and 'stablecoins', means every p

In [22]:
import pandas as pd

In [23]:
df = pd.DataFrame()

In [24]:
for post in req.json()['data']['children']:
    new_post_df = pd.DataFrame([{
        'subreddit': post['data']['subreddit'],
        'title': post['data']['title'],
        'selftext': post['data']['selftext'],
        'created_utc': post['data']['created_utc']
    }])
    df = pd.concat([df, new_post_df], ignore_index=True)

In [25]:
df.head()

,subreddit,title,selftext,created_utc
0,Bitcoin,"Daily Discussion, November 21, 2022",Please utilize this sticky thread for all gene...,1.669007e+09
1,Bitcoin,"Mentor Monday, November 21, 2022: Ask all your...",Ask (and answer!) away! Here are the general r...,1.669011e+09
2,Bitcoin,Why are humans so naive?,,1.669027e+09
3,Bitcoin,Tucker on Monetary Printing Presses,,1.669047e+09
4,Bitcoin,Too soon?? New Bitcoin piece features a live t...,,1.669010e+09


### Streaming New Posts

In [26]:
listing = "new"

In [27]:
req = requests.get(f"https://oauth.reddit.com/r/{subreddit}/{listing}", headers=headers, params={"limit": "100"})
req

<Response [200]>

In [28]:
df = pd.DataFrame()
for post in req.json()['data']['children']:
    new_post_df = pd.DataFrame([{
        'post_id': f"{post['kind']}_{post['data']['id']}",
        'subreddit': post['data']['subreddit'],
        'title': post['data']['title'],
        'selftext': post['data']['selftext'],
        'created_utc': post['data']['created_utc']
    }])
    df = pd.concat([df, new_post_df], ignore_index=True)

In [29]:
# post ID
post = req.json()['data']['children'][-1]
post_id = f"{post['kind']}_{post['data']['id']}"
post_id

't3_z05nyd'

### Looping back in time

In [30]:
req = requests.get(f"https://oauth.reddit.com/r/{subreddit}/{listing}", 
                   headers=headers, 
                   params={"limit": "100", "after": post_id})
req

<Response [200]>

In [31]:
for post in req.json()['data']['children']:
    new_post_df = pd.DataFrame([{
        'post_id': f"{post['kind']}_{post['data']['id']}",
        'subreddit': post['data']['subreddit'],
        'title': post['data']['title'],
        'selftext': post['data']['selftext'],
        'created_utc': post['data']['created_utc']
    }])
    df = pd.concat([df, new_post_df], ignore_index=True)

In [32]:
df

,post_id,subreddit,title,selftext,created_utc
0,t3_z19upj,Bitcoin,"Today Elizabeth Warren, Dick Durbin, and Tina ...",,1.669064e+09
1,t3_z19rkc,Bitcoin,Wholecoiner brag post,"I know these kinds of posts are lame, but I di...",1.669064e+09
2,t3_z19l2a,Bitcoin,Ask not how the state will regulate Bitcoin. A...,,1.669063e+09
3,t3_z19hkd,Bitcoin,This is showing red for the last couple of day...,&amp;#x200B;\n\nhttps://preview.redd.it/kttb64...,1.669063e+09
4,t3_z18v31,Bitcoin,Please address my perceived weaknesses of Bitcoin,"First of all, I believe in the future of Bitco...",1.669061e+09
...,...,...,...,...,...
195,t3_yz43au,Bitcoin,"Daily Discussion, November 19, 2022",Please utilize this sticky thread for all gene...,1.668834e+09
196,t3_yz3lrm,Bitcoin,General public attitude and awareness,I've been following along and reading for a wh...,1.668833e+09
197,t3_yz3lh1,Bitcoin,Grayscale refuses to show proof of reserves be...,,1.668833e+09
198,t3_yz329t,Bitcoin,Never trust...,"Never trust a guy called ""Bank Fried"" it fores...",1.668831e+09
